<a href="https://colab.research.google.com/github/garvsharmxa/3D_Model_Reconstructions/blob/main/3Dify_Image_to_Dimension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **3D Model Reconstruction For Blender**:
![](https://shunsukesaito.github.io/PIFuHD/resources/images/pifuhd.gif)






## **Note**
Make sure that your runtime type is 'Python 3 with GPU acceleration'. To do so, go to Edit > Notebook settings > Hardware Accelerator > Select "GPU".

## **Requirements**
- Python 3
- IPython
- PyTorch
- PIFuHD
- Torch
- TorchVision
- json
- PIL
- skimage
- numpy
- cv2

## **Help! I'm new to Google Colab**

You can check out the following youtube video on how to upload your own picture and run GHD. **Note that with new update, you can upload your own picture more easily with GUI down below.**


In [1]:
!pip3 install --upgrade pip

#@title STEP1: Execute to Setup Pifuhd
!git clone https://github.com/facebookresearch/pifuhd
!git clone https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch.git

%cd /content/lightweight-human-pose-estimation.pytorch/
!wget https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estmation/checkpoint_iter_370000.pth

%cd /content/pifuhd/
!sh ./scripts/download_trained_model.sh


!pip install 'torch==2.4.1' -f https://download.pytorch.org/whl/torch_stable.html
!pip install 'torchvision==0.19.1' -f https://download.pytorch.org/whl/torch_stable.html
!pip install 'git+https://github.com/facebookresearch/pytorch3d.git'



%cd /content/lightweight-human-pose-estimation.pytorch/
import torch
import cv2
import numpy as np
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
import demo
from IPython.display import clear_output

def get_rect(net, images, height_size):
    net = net.eval()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)

            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
              pmin = valid_keypoints.min(0)
              pmax = valid_keypoints.max(0)

              center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int)
              radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
              # if leg is missing, use pelvis to get cropping
              center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int)
              radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
              center[1] += int(0.05*radius)
            else:
              center = np.array([img.shape[1]//2,img.shape[0]//2])
              radius = max(img.shape[1]//2,img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius

            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Cloning into 'pifuhd'...
remote: Enumerating objects: 222, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 222 (delta 20), reused 20 (delta 20), pack-reused 200 (from 1)
Receiving objects: 100% (222/222), 399.83 KiB | 11.42 MiB/s, done.
Resolving deltas: 100% (113/113), done.
Cloning into 'lightweight-human-pose-estimation.pytorch'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 124 (delta 20), reused 18 (delta 18), pack-reused 91 (from 2)
Receiving objects: 100% (124/124), 230.37 KiB | 6.78 MiB/s, done.
Resolving deltas: 100% (52/52), done.
/content/lightweight-human-pose-estimation.pytorch
--2025-07-30

# **Using Form Internet**



In [ ]:
#@title **Uploadind Image From Internet**
#@markdown * Find an image on the Internet
#@markdown * Right click and copy image address
#@markdown * Paste the image address to image_url
image_url = 'https://toppng.com/uploads/preview/transparent-human-11550225796ipkgyynyvt.png' #@param {type:"string"}
path = '/content/pifuhd/sample_images/webImg'

#empty the folder
!rm -rf '/content/pifuhd/sample_images'
!mkdir '/content/pifuhd/sample_images'

!wget {image_url} -O {path}
import cv2
img = cv2.imread(path)
cv2.imwrite('/content/pifuhd/sample_images/webImg.png', img)
!rm -f '$path'

import os
image_path = '/content/pifuhd/sample_images/webImg.png'
image_dir = os.path.dirname(image_path)
file_name = os.path.splitext(os.path.basename(image_path))[0]

# output pathes
obj_path = '/content/pifuhd/results/pifuhd_final/recon/result_%s_256.obj' % file_name
out_img_path = '/content/pifuhd/results/pifuhd_final/recon/result_%s_256.png' % file_name
video_path = '/content/pifuhd/results/pifuhd_final/recon/result_%s_256.mp4' % file_name
video_display_path = '/content/pifuhd/results/pifuhd_final/result_%s_256_display.mp4' % file_name

# Make sure we're in the right directory and download the checkpoint
%cd /content/lightweight-human-pose-estimation.pytorch/

def get_rect(net, images, height_size):
    net = net.eval()
    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33

    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale

        current_poses = []
        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue

            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1  # Fixed: np.int -> np.int32
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])

            valid_keypoints = np.array(valid_keypoints)
            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
                pmin = valid_keypoints.min(0)
                pmax = valid_keypoints.max(0)
                center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int32)  # Fixed: np.int -> np.int32
                radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
                # if leg is missing, use pelvis to get cropping
                center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int32)  # Fixed: np.int -> np.int32
                radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
                center[1] += int(0.05*radius)
            else:
                center = np.array([img.shape[1]//2, img.shape[0]//2])
                radius = max(img.shape[1]//2, img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius
            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')

# Check if checkpoint exists, if not download it
import os
checkpoint_path = '/content/lightweight-human-pose-estimation.pytorch/checkpoint_iter_370000.pth'

if not os.path.exists(checkpoint_path):
    print("Downloading checkpoint file...")
    !wget https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth
else:
    print("Checkpoint file already exists")

# Now load the network with checkpoint
net = PoseEstimationWithMobileNet()
checkpoint = torch.load('checkpoint_iter_370000.pth', map_location='cpu')
load_state(net, checkpoint)
get_rect(net.cuda(), [image_path], 512)

#run
%cd /content/pifuhd/
# Warning: all images with the corresponding rectangle files under -i will be processed.
!python -m apps.simple_test -r 256 --use_rect -i $image_dir
# seems that 256 is the maximum resolution that can fit into Google Colab.
# If you want to reconstruct a higher-resolution mesh, please try with your own machine.

#clear everything
clear_output()
#render video
from lib.colab_util import generate_video_from_obj, set_renderer, video

renderer = set_renderer()
generate_video_from_obj(obj_path, out_img_path, video_path, renderer)

# we cannot play a mp4 video generated by cv2
!ffmpeg -i $video_path -vcodec libx264 $video_display_path -y -loglevel quiet
video(video_display_path)

In [3]:
#@title **Download 3D Model (.obj file) and Results (From Internet Image)**
import os
from google.colab import files

# Define paths for internet-sourced image results
RESULTS_PATH = '/content/pifuhd/results/pifuhd_final/recon'
file_name = 'webImg'  # Standard naming for internet-downloaded images

# Define all possible output files
obj_path = f'{RESULTS_PATH}/result_{file_name}_256.obj'
png_path = f'{RESULTS_PATH}/result_{file_name}_256.png'
video_path = f'{RESULTS_PATH}/result_{file_name}_256.mp4'
video_display_path = f'/content/pifuhd/results/pifuhd_final/result_{file_name}_256_display.mp4'

print("🌐 Downloading results from internet image processing...")
print("Checking available files...")

# Check which files exist and prepare for download
files_to_download = []

if os.path.exists(obj_path):
    files_to_download.append(('3D Model (.obj)', obj_path))
    print(f"✓ 3D model found: result_{file_name}_256.obj")
else:
    print(f"✗ 3D model not found: result_{file_name}_256.obj")

if os.path.exists(png_path):
    files_to_download.append(('Rendered Image (.png)', png_path))
    print(f"✓ Rendered image found: result_{file_name}_256.png")
else:
    print(f"✗ Rendered image not found: result_{file_name}_256.png")

if os.path.exists(video_display_path):
    files_to_download.append(('3D Animation (.mp4)', video_display_path))
    print(f"✓ 3D animation found: result_{file_name}_256_display.mp4")
elif os.path.exists(video_path):
    files_to_download.append(('3D Animation (.mp4)', video_path))
    print(f"✓ 3D animation found: result_{file_name}_256.mp4")
else:
    print(f"✗ 3D animation not found")

# Get file size information for the main .obj file
if os.path.exists(obj_path):
    file_size = os.path.getsize(obj_path)
    file_size_mb = file_size / (1024 * 1024)
    print(f"📊 3D model size: {file_size_mb:.2f} MB")

# Download all available files
if files_to_download:
    print(f"\n📥 Downloading {len(files_to_download)} file(s)...")

    for file_desc, file_path in files_to_download:
        try:
            file_name_only = os.path.basename(file_path)
            print(f"Downloading {file_desc}: {file_name_only}...")
            files.download(file_path)
            print(f"✅ Successfully downloaded: {file_name_only}")
        except Exception as e:
            print(f"❌ Failed to download {file_desc}: {str(e)}")

    print(f"\n🎉 Download process completed!")

else:
    print("\n❌ No output files found! The internet image processing may not have completed successfully.")
    print("\n🔧 Troubleshooting steps:")
    print("1. Verify the internet image processing script ran completely")
    print("2. Check if the image URL was valid and accessible")
    print("3. Ensure the pose estimation detected a human figure")
    print("4. Confirm PIFuHD processing completed without errors")

    # Enhanced debugging - show directory structure
    print(f"\n📁 Checking results directory structure:")

    if os.path.exists('/content/pifuhd/results/'):
        print("Results directory exists. Contents:")
        for root, dirs, files in os.walk('/content/pifuhd/results/'):
            level = root.replace('/content/pifuhd/results/', '').count(os.sep)
            indent = ' ' * 2 * level
            print(f'{indent}{os.path.basename(root)}/')
            subindent = ' ' * 2 * (level + 1)
            for file in files:
                if file.endswith(('.obj', '.png', '.mp4')):
                    print(f'{subindent}📄 {file}')
    else:
        print("❌ Results directory not found!")

    # Check sample images directory
    if os.path.exists('/content/pifuhd/sample_images/'):
        print(f"\n📁 Sample images directory contents:")
        for file in os.listdir('/content/pifuhd/sample_images/'):
            print(f"  📄 {file}")

print("\n" + "="*60)
print("🌐 Internet Image 3D Model Download Complete!")
print("="*60)
print("💡 Tip: You can view .obj files in 3D modeling software like:")
print("   • Blender (free)")
print("   • MeshLab (free)")
print("   • 3D Viewer (Windows)")
print("   • Online viewers like viewstl.com")
print("="*60)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Using From Desktop**

In [ ]:
#@title **Upload Image From Local Desktop**
import os
import cv2
import numpy as np
import torch
from google.colab import files
from IPython.display import clear_output

# Step 1: Set up paths
SAMPLE_IMAGES_PATH = '/content/pifuhd/sample_images'
RESULTS_PATH = '/content/pifuhd/results/pifuhd_final/recon'
CHECKPOINT_PATH = '/content/lightweight-human-pose-estimation.pytorch/checkpoint_iter_370000.pth'

# Clean up previous results
print("Cleaning up previous results...")
os.system(f'rm -rf {SAMPLE_IMAGES_PATH}')
os.makedirs(SAMPLE_IMAGES_PATH, exist_ok=True)

# Step 2: Upload Image
print("Upload your image file")
uploaded = files.upload()

if not uploaded:
    raise ValueError("No file uploaded. Please upload an image.")

filename = list(uploaded.keys())[0]
image_path = f"./{filename}"

# Check if file exists and can be read
if not os.path.exists(image_path):
    raise FileNotFoundError(f"Uploaded file not found at {image_path}")

# Save uploaded image to the sample images folder
img = cv2.imread(image_path)

if img is None:
    raise ValueError(f"Could not read image from {image_path}. Please check if it's a valid image file.")

# Save with consistent naming
target_image_path = f"{SAMPLE_IMAGES_PATH}/webImg.png"
cv2.imwrite(target_image_path, img)

# Clean up original uploaded file
os.remove(image_path)

# Update paths to use the consistent naming
image_path = target_image_path
image_dir = os.path.dirname(image_path)
file_name = os.path.splitext(os.path.basename(image_path))[0]

# Output paths
obj_path = f"{RESULTS_PATH}/result_{file_name}_256.obj"
out_img_path = f"{RESULTS_PATH}/result_{file_name}_256.png"
video_path = f"{RESULTS_PATH}/result_{file_name}_256.mp4"
video_display_path = f"/content/pifuhd/results/pifuhd_final/result_{file_name}_256_display.mp4"

# Step 3: Setup and download checkpoint if needed
print("Setting up Lightweight Human Pose Estimation...")
os.chdir('/content/lightweight-human-pose-estimation.pytorch/')

# Check if checkpoint exists, if not download it
if not os.path.exists(CHECKPOINT_PATH):
    print("Downloading checkpoint file...")
    os.system('wget https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth')
else:
    print("Checkpoint file already exists")

# Import required modules
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.load_state import load_state
from modules.keypoints import extract_keypoints, group_keypoints
from modules.pose import Pose
import demo

# Fixed get_rect function with proper numpy compatibility
def get_rect(net, images, height_size):
    net = net.eval()
    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts

    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale

        current_poses = []
        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue

            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1  # Fixed: np.int -> np.int32
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])

            valid_keypoints = np.array(valid_keypoints)
            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
                pmin = valid_keypoints.min(0)
                pmax = valid_keypoints.max(0)
                center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int32)  # Fixed: np.int -> np.int32
                radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
                # if leg is missing, use pelvis to get cropping
                center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int32)  # Fixed: np.int -> np.int32
                radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
                center[1] += int(0.05*radius)
            else:
                center = np.array([img.shape[1]//2, img.shape[0]//2])
                radius = max(img.shape[1]//2, img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius
            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')

# Load the network with checkpoint
print("Loading pose estimation model...")
net = PoseEstimationWithMobileNet()
checkpoint = torch.load(CHECKPOINT_PATH, map_location='cpu')
load_state(net, checkpoint)

# Run pose estimation
print("Running pose estimation...")
get_rect(net.cuda(), [image_path], 512)

# Step 4: Run PIFuHD
print("Running PIFuHD for 3D reconstruction...")
os.chdir('/content/pifuhd/')
os.makedirs(RESULTS_PATH, exist_ok=True)

# Warning: all images with the corresponding rectangle files under -i will be processed.
os.system(f'python -m apps.simple_test -r 256 --use_rect -i {image_dir}')
# 256 is the maximum resolution that can fit into Google Colab.
# If you want to reconstruct a higher-resolution mesh, please try with your own machine.

# Step 5: Generate video output
print("Generating video output...")
clear_output()

# Import video generation utilities
from lib.colab_util import generate_video_from_obj, set_renderer, video

# Render video
renderer = set_renderer()
generate_video_from_obj(obj_path, out_img_path, video_path, renderer)

# Convert video for display (we cannot play a mp4 video generated by cv2)
os.system(f'ffmpeg -i {video_path} -vcodec libx264 {video_display_path} -y -loglevel quiet')

# Display the final video
print("3D reconstruction complete! Playing result video...")
video(video_display_path)

In [ ]:
#@title **Download 3D Model (.obj file) and Results**
import os
from google.colab import files

# Define paths
RESULTS_PATH = '/content/pifuhd/results/pifuhd_final/recon'
file_name = 'webImg'  # This should match the naming from the processing script

# Define all possible output files
obj_path = f'{RESULTS_PATH}/result_{file_name}_256.obj'
png_path = f'{RESULTS_PATH}/result_{file_name}_256.png'
video_path = f'{RESULTS_PATH}/result_{file_name}_256.mp4'
video_display_path = f'/content/pifuhd/results/pifuhd_final/result_{file_name}_256_display.mp4'

print("Checking available files...")

# Check which files exist and download them
files_to_download = []

if os.path.exists(obj_path):
    files_to_download.append(('3D Model (.obj)', obj_path))
    print(f"✓ 3D model found: {obj_path}")
else:
    print(f"✗ 3D model not found: {obj_path}")

if os.path.exists(png_path):
    files_to_download.append(('Rendered Image (.png)', png_path))
    print(f"✓ Rendered image found: {png_path}")
else:
    print(f"✗ Rendered image not found: {png_path}")

if os.path.exists(video_display_path):
    files_to_download.append(('3D Animation (.mp4)', video_display_path))
    print(f"✓ 3D animation found: {video_display_path}")
elif os.path.exists(video_path):
    files_to_download.append(('3D Animation (.mp4)', video_path))
    print(f"✓ 3D animation found: {video_path}")
else:
    print(f"✗ 3D animation not found")

# Download all available files
if files_to_download:
    print(f"\nDownloading {len(files_to_download)} file(s)...")
    for file_desc, file_path in files_to_download:
        try:
            print(f"Downloading {file_desc}...")
            files.download(file_path)
            print(f"✓ Successfully downloaded: {os.path.basename(file_path)}")
        except Exception as e:
            print(f"✗ Failed to download {file_desc}: {str(e)}")
else:
    print("\n❌ No output files found! Please make sure the PIFuHD processing completed successfully.")
    print("\nTroubleshooting:")
    print("1. Check if the processing script ran without errors")
    print("2. Verify the image was uploaded and processed correctly")
    print("3. Make sure the pose estimation step completed successfully")

    # List all files in the results directory for debugging
    if os.path.exists(RESULTS_PATH):
        print(f"\nFiles found in {RESULTS_PATH}:")
        for file in os.listdir(RESULTS_PATH):
            print(f"  - {file}")
    else:
        print(f"\nResults directory does not exist: {RESULTS_PATH}")

print("\n" + "="*50)
print("Download complete!")
print("="*50)

# **Refrence**

github:- https://github.com/garvsharmxa/

Methods refrence's:- https://shunsukesaito.github.io/PIFuHD/

Background Remover :- https://www.remove.bg/

